# Second Quantization Examples
Link to Second quant documentation: https://docs.sympy.org/latest/modules/physics/secondquant.html

In [1]:
import sys

from sympy.physics.secondquant import (AntiSymmetricTensor, wicks,
        F, Fd, NO, evaluate_deltas, substitute_dummies, Commutator,
        simplify_index_permutations, PermutationOperator, contraction)
from sympy import (
    symbols, expand, pprint, Rational, latex, Dummy
)



pretty_dummies_dict = {
    'above': 'abcdefgh',
    'below': 'ijklmno',
    'general': 'pqrstu'
}


p, q, r, s = symbols('p,q,r,s', cls=Dummy)

## Setup creation and annihilation operators

In [2]:
from IPython.display import Markdown
ap_dagger  = Fd(p)
aq		   = F(q)

## Perform a contraction

In [3]:
contr      = evaluate_deltas(contraction(ap_dagger,aq))
Markdown("""
$${} = {}$$
""".format('{:}'+latex(ap_dagger)+latex(aq) + '{:}',latex(contr)))


$${:}a^\dagger_{p}a_{q}{:} = \delta_{i p}$$


## Setup Hamiltonian, not on normal order form

In [4]:
h  = AntiSymmetricTensor('h', (p,), (q,))
pq = ap_dagger*aq

V    = AntiSymmetricTensor('V', (p, q), (r, s))
pqsr = Fd(p)*Fd(q)*F(s)*F(r)

H0 = h*pq
HI = Rational(1, 4)*V*pqsr

H = H0+HI
Markdown("$${}$$".format(latex(H)))

$$\frac{V^{pq}_{rs} a^\dagger_{p} a^\dagger_{q} a_{s} a_{r}}{4} + h^{p}_{q} a^\dagger_{p} a_{q}$$

## Compute the normal ordered form of the Hamiltonian
`sympy.physics.secondquant.wicks(e, **kw_args)` returns the normal ordered equivalent of an expression using Wicks Theorem

In [5]:
H_N  = evaluate_deltas(wicks(H))
H_N  = substitute_dummies(H_N, new_indices=True, pretty_indices=pretty_dummies_dict)

Eref = evaluate_deltas(wicks(H, keep_only_fully_contracted=True))
Eref = substitute_dummies(Eref, new_indices=True, pretty_indices=pretty_dummies_dict)

Markdown("""
$$E_{{ref}} = {}$$ 
$$H = {}$$
""".format(latex(Eref), latex(H_N)))


$$E_{ref} = \frac{V^{ij}_{ij}}{2} + h^{i}_{i}$$ 
$$H = \frac{V^{ij}_{ij}}{2} + V^{iq}_{ip} \left\{a^\dagger_{q} a_{p}\right\} - \frac{V^{rs}_{pq} \left\{a^\dagger_{r} a^\dagger_{s} a_{p} a_{q}\right\}}{4} + h^{i}_{i} + h^{q}_{p} \left\{a^\dagger_{q} a_{p}\right\}$$


## Setup Hamiltonian on normal ordered form

In [6]:
E0 = symbols('Eref', real = True, constant = True) #Reference energy

f  = AntiSymmetricTensor('f', (p,), (q,))
pq = NO(ap_dagger*aq)

V    = AntiSymmetricTensor('V', (p, q), (r, s))
pqsr = NO(Fd(p)*Fd(q)*F(s)*F(r)) 

HI = Rational(1, 4)*V*pqsr
Fock = f*pq #F is reserved by sympy

HN = E0+Fock+HI
Markdown("""
$$H = {}$$
""".format(latex(HN)))


$$H = Eref - \frac{V^{pq}_{rs} \left\{a^\dagger_{p} a^\dagger_{q} a_{r} a_{s}\right\}}{4} + f^{p}_{q} \left\{a^\dagger_{p} a_{q}\right\}$$


# Compute expectations
## Define indices above and below Fermi level

In [7]:
i, j, k, l = symbols('i,j,k,l', below_fermi=True)
a, b, c, d = symbols('a,b,c,d', above_fermi=True)

# Compute $\langle c|H_N|c\rangle $  

In [8]:
cHc = wicks(HN, simplify_kronecker_deltas=True, keep_only_fully_contracted=True)
Markdown(r"$$\langle c|H_{{normal}}|c \rangle = {} $$".format(latex(cHc)))

$$\langle c|H_{normal}|c \rangle = Eref $$

# Compute $\langle c|F|\Phi_i^a\rangle $  

In [9]:
cFphi_ia = wicks((Fock)*NO(Fd(a)*F(i)), simplify_kronecker_deltas=True, keep_only_fully_contracted=True)
cFphi_ia = substitute_dummies(cFphi_ia)

Markdown(r"$$\langle c|F|\Phi_i^a \rangle = {} $$".format(latex(cFphi_ia)))

$$\langle c|F|\Phi_i^a \rangle = f^{i}_{a} $$